In [1]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import matplotlib.pyplot as plt

### Training Model

In [2]:
def pre_train_process(df,startIdx,windowLength,isNormalized):
    tmp = df.copy()
    tmp.reset_index(inplace=True)
    train_start_index = startIdx
    train_window_length = windowLength
    train_end_index = train_start_index+train_window_length-1
    tmp = tmp.iloc[train_start_index:train_end_index]
    ## normalize the option price change and future price change
    if(isNormalized==False):
        tmp['opt_price_chg'] = tmp['opt_price'].diff()
        tmp['fut_price_chg'] = tmp['fut_price'].diff()
        tmp = tmp[tmp['fut_price_chg']!=0]
    else:
        tmp['opt_price_chg'] = tmp['opt_price'] / tmp['opt_price'].shift()
        tmp['fut_price_chg'] = tmp['fut_price'] / tmp['fut_price'].shift()
        tmp = tmp[tmp['fut_price_chg']!=1]
    tmp = tmp[['ts','opt_price_chg','fut_price_chg','delta','vega','days_to_expiry','fut_price']]
    tmp.reset_index(drop=True,inplace=True)
    train_set = pd.DataFrame()
    coeff = tmp['vega']*tmp['fut_price_chg']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    train_set['y'] = (tmp['opt_price_chg'] - tmp['delta']*tmp['fut_price_chg'])
    train_set['x1'] = coeff*tmp['delta']
    train_set['x2'] = coeff*tmp['delta']**2
    train_set.drop(index=0,inplace=True)
    return train_set

In [3]:
def train_model(train_set):
    x = sm.add_constant(train_set.iloc[:,1:])
    y = train_set['y']
    model = sm.OLS(y,x)
    result = model.fit()
    return result

In [4]:
def predict_MV_delta(df,startIdx,windowLength,result):
    tmp = df.copy()
    test_start_index = startIdx
    test_window_length = windowLength
    test_end_index = test_start_index + test_window_length-1
    tmp = tmp.iloc[test_start_index:test_end_index]
    tmp = tmp[['ts','delta','vega','days_to_expiry','fut_price']]
    b0 = result.params[0]
    b1 = result.params[1]
    b2 = result.params[2]
    tmp['y_hat'] = b0+b1*tmp['delta']+b2*tmp['delta']**2
    tmp['MV_delta'] = tmp['delta']+tmp['y_hat']*tmp['vega']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    tmp.reset_index(inplace=True)
    return tmp

### Read in data

In [142]:
Calls = pd.read_csv('nifty_311221_190122_1min_calls_py_vollib.csv')
contractList = Calls['opt_symbol'].unique()
#contractList

In [6]:
def generate_MV_delta(contractName,trainStartIdx,trainLength,isNormalized,predictMVdeltaLength):
    contract = Calls[Calls['opt_symbol']==contractName]
    train_set = pre_train_process(contract,trainStartIdx,trainLength,isNormalized)
    result = train_model(train_set)
    print("result rsquared: ",result.rsquared)
    predict_set = predict_MV_delta(contract,trainStartIdx+trainLength,predictMVdeltaLength,result)
    return predict_set

In [175]:
def generate_MV_delta2(contractName,Date,isNormalized,threshold):
    contract = Calls[Calls['opt_symbol']==contractName].copy()
    contract.reset_index(inplace=True,drop=True)
    idx = contract.index[contract['ts']<=Date].tolist()[-1]
    print("available data rows:", idx)
    i = 500
    rsquare = 0
    while(rsquare<threshold and i <= idx):
        train_set = pre_train_process(contract,idx-i,i,isNormalized)
        result = train_model(train_set)
        rsquare = result.rsquared
        i += 10
    if rsquare < threshold:
        print('not enough data to make the regression model stable!')
        return
    print('i: ',i)
    print("result rsquared: ",result.rsquared)
    predict_set = predict_MV_delta(contract,idx,2,result)
    return predict_set

In [184]:
contract = 'NIFTY22JAN17250CE'
tmp = Calls[Calls['opt_symbol']==contract]
NIFTY22JAN17250CE = generate_MV_delta2(contract,'2022-01-10 15:29:00',True,0.95)
#NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 2622
i:  630
result rsquared:  0.9500447096727037


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,2622,2022-01-10 15:29:00,0.884482,276.153136,17.001042,18054.95,-2.518921,0.736152


In [24]:
contract = 'NIFTY22JAN15850CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,37,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 47
result rsquared:  0.999598497260737


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,294529,2022-01-18 15:14:00,0.880935,129.535738,9.011458,18110.65,-8.248251,0.568960
1,294670,2022-01-18 15:15:00,0.879680,128.958310,9.010764,18108.25,-8.248981,0.569014
2,294687,2022-01-18 15:16:00,0.879810,129.012753,9.010069,18108.50,-8.248907,0.569008
3,294914,2022-01-18 15:19:00,0.888857,133.097220,9.007986,18125.25,-8.242782,0.568713
4,294971,2022-01-18 15:20:00,0.888354,132.865516,9.007292,18124.35,-8.243174,0.568724
5,295340,2022-01-18 15:25:00,0.892037,134.493437,9.003819,18130.85,-8.240167,0.568662
6,295386,2022-01-18 15:26:00,0.891577,134.282515,9.003125,18130.05,-8.240560,0.568668
7,295471,2022-01-18 15:27:00,0.887769,132.566350,9.002431,18123.30,-8.243621,0.568737
8,295584,2022-01-18 15:28:00,0.885540,131.555082,9.001736,18119.25,-8.245253,0.568796


In [31]:
contract = 'NIFTY22JAN16000CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,1000,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 3191
result rsquared:  0.9534433481471027


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,66755,2022-01-04 14:24:00,0.985301,296.778331,23.046181,17779.4,-0.340876,0.966485
1,66816,2022-01-04 14:25:00,0.973603,291.328318,23.045486,17778.0,-0.122745,0.966951
2,66885,2022-01-04 14:26:00,0.984183,296.290903,23.044792,17775.0,-0.319814,0.966554
3,66955,2022-01-04 14:27:00,0.974484,291.878139,23.044097,17770.5,-0.139005,0.966934
4,67008,2022-01-04 14:28:00,0.995433,302.485000,23.043403,17779.0,-0.533950,0.965391
5,67070,2022-01-04 14:29:00,0.992172,300.435466,23.042708,17778.0,-0.471396,0.965828
6,67224,2022-01-04 14:31:00,0.976746,292.549925,23.041319,17786.5,-0.180848,0.966909
7,67238,2022-01-04 14:32:00,0.970774,289.906367,23.040625,17783.4,-0.070774,0.966958
8,67302,2022-01-04 14:33:00,0.967906,288.662353,23.039931,17781.8,-0.018380,0.966919


In [34]:
contract = 'NIFTY22JAN16250CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,225,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 235
result rsquared:  0.533144394463016


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,228082,2022-01-13 14:50:00,0.974005,227.376422,14.028125,18255.45,0.259622,0.987710
1,228155,2022-01-13 14:51:00,0.966245,224.134835,14.027431,18250.60,0.405066,0.987330
2,228213,2022-01-13 14:52:00,0.970046,225.698559,14.026736,18253.05,0.334120,0.987557
3,228254,2022-01-13 14:53:00,0.968390,225.006160,14.026042,18252.00,0.365099,0.987468
4,228325,2022-01-13 14:54:00,0.961368,222.135074,14.025347,18247.25,0.495286,0.986925
5,228445,2022-01-13 14:55:00,0.966095,224.050949,14.024653,18250.50,0.407866,0.987320
6,228456,2022-01-13 14:56:00,0.968390,224.989449,14.023958,18252.00,0.365099,0.987468
7,228512,2022-01-13 14:57:00,0.977070,228.658303,14.023264,18257.20,0.201504,0.987768
8,228623,2022-01-13 14:58:00,0.970850,226.000417,14.022569,18253.55,0.319041,0.987595


In [39]:
contract = 'NIFTY22JAN16500CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,600,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 4465
result rsquared:  0.9524335881994076


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,37025,2022-01-03 13:00:00,0.911743,308.348628,24.104514,17612.00,-2.008681,0.798034
1,37031,2022-01-03 13:01:00,0.906790,307.192021,24.103819,17614.05,-1.916923,0.798693
2,37135,2022-01-03 13:02:00,0.904655,306.760832,24.103125,17612.60,-1.877639,0.798912
3,37168,2022-01-03 13:03:00,0.905115,306.880957,24.102431,17612.00,-1.886081,0.798850
4,37249,2022-01-03 13:04:00,0.909444,307.735537,24.101736,17614.90,-1.965981,0.798385
5,37331,2022-01-03 13:05:00,0.910207,307.880986,24.101042,17615.40,-1.980123,0.798298
6,37409,2022-01-03 13:06:00,0.907693,307.344320,24.100347,17614.80,-1.933582,0.798599
7,37424,2022-01-03 13:07:00,0.907108,307.262132,24.099653,17613.30,-1.922785,0.798642
8,37524,2022-01-03 13:08:00,0.905784,307.078797,24.098958,17610.00,-1.898391,0.798736


In [44]:
contract = 'NIFTY22JAN16750CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,200,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 2402
result rsquared:  0.9803488078135688


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,14518,2021-12-31 13:14:00,0.832502,334.309322,27.094792,17442.20,-4.269010,0.582967
1,14575,2021-12-31 13:15:00,0.833593,334.445134,27.094097,17443.00,-4.287402,0.582889
2,14625,2021-12-31 13:16:00,0.836353,334.792127,27.093403,17445.00,-4.334111,0.582681
3,14711,2021-12-31 13:17:00,0.833251,334.392758,27.092708,17442.75,-4.281637,0.582914
4,14772,2021-12-31 13:18:00,0.809648,330.365482,27.092014,17438.00,-3.890820,0.584836
5,14791,2021-12-31 13:19:00,0.807768,330.076942,27.091319,17436.35,-3.860336,0.584889
6,14911,2021-12-31 13:20:00,0.807429,330.021093,27.090625,17436.05,-3.854842,0.584897
7,14917,2021-12-31 13:21:00,0.801822,329.151184,27.089931,17431.00,-3.764540,0.585011
8,14996,2021-12-31 13:22:00,0.799924,328.848342,27.089236,17429.25,-3.734168,0.585036


In [46]:
contract = 'NIFTY22JAN17000CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,500,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5129
result rsquared:  0.9543982841700841


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,30666,2022-01-03 11:20:00,0.783117,319.287324,24.173958,17567.80,-4.331930,0.528919
1,30747,2022-01-03 11:21:00,0.778883,318.606379,24.173264,17566.85,-4.254024,0.529772
2,30845,2022-01-03 11:22:00,0.775784,318.282321,24.172569,17563.00,-4.197356,0.530184
3,30858,2022-01-03 11:23:00,0.775475,318.407001,24.171875,17560.00,-4.191721,0.530063
4,30965,2022-01-03 11:24:00,0.775421,318.269357,24.171181,17562.00,-4.190743,0.530197
5,31016,2022-01-03 11:25:00,0.780932,318.878494,24.170486,17568.00,-4.291663,0.529404
6,31053,2022-01-03 11:26:00,0.779862,318.615471,24.169792,17569.20,-4.272001,0.529707
7,31131,2022-01-03 11:27:00,0.780715,318.760464,24.169097,17569.20,-4.287663,0.529524
8,31153,2022-01-03 11:28:00,0.776434,318.131992,24.168403,17566.90,-4.209218,0.530289


In [49]:
contract = 'NIFTY22JAN17250CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,600,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 4482
result rsquared:  0.9685772782462364


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,37006,2022-01-03 13:00:00,0.696722,322.749772,24.104514,17612.00,-7.451915,0.255176
1,37050,2022-01-03 13:01:00,0.698612,322.958860,24.103819,17614.05,-7.485689,0.254822
2,37109,2022-01-03 13:02:00,0.697273,322.803028,24.103125,17612.60,-7.461755,0.255073
3,37211,2022-01-03 13:03:00,0.696722,322.735824,24.102431,17612.00,-7.451915,0.255176
4,37259,2022-01-03 13:04:00,0.697398,322.606396,24.101736,17614.90,-7.463989,0.255380
5,37343,2022-01-03 13:05:00,0.697856,322.654141,24.101042,17615.40,-7.472171,0.255294
6,37364,2022-01-03 13:06:00,0.697307,322.586623,24.100347,17614.80,-7.462356,0.255397
7,37462,2022-01-03 13:07:00,0.695941,322.424810,24.099653,17613.30,-7.437980,0.255652
8,37514,2022-01-03 13:08:00,0.692975,322.074445,24.098958,17610.00,-7.385141,0.256201


In [50]:
contract = 'NIFTY22JAN17500CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,600,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5249
result rsquared:  0.959157437842517


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,36999,2022-01-03 13:00:00,0.571044,326.834605,24.104514,17612.00,-7.081135,0.146157
1,37040,2022-01-03 13:01:00,0.572307,326.922509,24.103819,17614.05,-7.102877,0.146045
2,37108,2022-01-03 13:02:00,0.571525,326.919159,24.103125,17612.60,-7.089403,0.146034
3,37173,2022-01-03 13:03:00,0.571291,326.969416,24.102431,17612.00,-7.085383,0.145955
4,37284,2022-01-03 13:04:00,0.572941,327.009683,24.101736,17614.90,-7.113782,0.145912
5,37347,2022-01-03 13:05:00,0.572900,326.823050,24.101042,17615.40,-7.113082,0.146163
6,37379,2022-01-03 13:06:00,0.572649,326.861488,24.100347,17614.80,-7.108766,0.146100
7,37454,2022-01-03 13:07:00,0.571735,326.794554,24.099653,17613.30,-7.093019,0.146175
8,37538,2022-01-03 13:08:00,0.570041,326.845609,24.098958,17610.00,-7.063881,0.146078


In [52]:
contract = 'NIFTY22JAN17750CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,1000,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5240
result rsquared:  0.9762410424492035


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,62852,2022-01-04 13:25:00,0.512389,325.607058,23.087153,17758.20,-7.101722,0.082186
1,62919,2022-01-04 13:26:00,0.512282,325.464117,23.086458,17758.00,-7.099873,0.082368
2,62999,2022-01-04 13:27:00,0.513466,325.654201,23.085764,17760.00,-7.120303,0.082105
3,63045,2022-01-04 13:28:00,0.512323,325.332004,23.085069,17758.05,-7.100586,0.082529
4,63131,2022-01-04 13:29:00,0.513475,325.465064,23.084375,17760.00,-7.120464,0.082343
5,63197,2022-01-04 13:30:00,0.516347,325.701925,23.083681,17764.80,-7.170049,0.082007
6,63284,2022-01-04 13:31:00,0.514549,325.458104,23.082986,17761.80,-7.138998,0.082334
7,63333,2022-01-04 13:32:00,0.512157,325.487096,23.082292,17757.80,-7.097723,0.082300
8,63386,2022-01-04 13:33:00,0.507980,325.511458,23.081597,17750.85,-7.025694,0.082280


In [53]:
contract = 'NIFTY22JAN18000CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,1000,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5252
result rsquared:  0.9564726559168534


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,62900,2022-01-04 13:25:00,0.359194,329.108849,23.087153,17758.20,-4.663627,0.073646
1,62932,2022-01-04 13:26:00,0.359308,329.060655,23.086458,17758.00,-4.665565,0.073676
2,62991,2022-01-04 13:27:00,0.360288,329.111212,23.085764,17760.00,-4.682177,0.073622
3,63091,2022-01-04 13:28:00,0.359638,328.997729,23.085069,17758.05,-4.671149,0.073710
4,63125,2022-01-04 13:29:00,0.360597,329.045903,23.084375,17760.00,-4.687425,0.073658
5,63193,2022-01-04 13:30:00,0.362703,329.220198,23.083681,17764.80,-4.723177,0.073496
6,63261,2022-01-04 13:31:00,0.360980,329.175716,23.082986,17761.80,-4.693919,0.073550
7,63305,2022-01-04 13:32:00,0.358421,329.164953,23.082292,17757.80,-4.650514,0.073591
8,63416,2022-01-04 13:33:00,0.354177,329.114792,23.081597,17750.85,-4.578624,0.073679


In [58]:
contract = 'NIFTY22JAN18250CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,3500,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5252
result rsquared:  0.9690883242346798


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,215704,2022-01-13 11:20:00,0.507214,267.209881,14.173958,18252.00,-7.241135,0.060219
1,215766,2022-01-13 11:21:00,0.504949,267.100960,14.173264,18249.00,-7.204459,0.060315
2,215827,2022-01-13 11:22:00,0.506819,267.111642,14.172569,18251.45,-7.234738,0.060347
3,215922,2022-01-13 11:23:00,0.512589,267.285321,14.171875,18259.00,-7.328163,0.060234
4,215975,2022-01-13 11:24:00,0.510938,267.154382,14.171181,18256.85,-7.301429,0.060390
5,215982,2022-01-13 11:25:00,0.513347,267.201382,14.170486,18260.00,-7.340435,0.060379
6,216065,2022-01-13 11:26:00,0.513119,267.202284,14.169792,18259.70,-7.336739,0.060359
7,216119,2022-01-13 11:27:00,0.511894,267.169570,14.169097,18258.10,-7.316897,0.060363
8,216159,2022-01-13 11:28:00,0.511816,267.119248,14.168403,18258.00,-7.315637,0.060434


In [60]:
contract = 'NIFTY22JAN18500CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,3750,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5252
result rsquared:  0.9577529662525627


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,230416,2022-01-14 09:15:00,0.254863,260.425415,13.260764,18175.85,-3.100311,0.061217
1,230466,2022-01-14 09:16:00,0.246473,260.380271,13.260069,18165.00,-2.966045,0.061129
2,230511,2022-01-14 09:17:00,0.236527,260.250829,13.259375,18150.10,-2.806827,0.061071
3,230530,2022-01-14 09:18:00,0.241724,260.345276,13.258681,18158.85,-2.890032,0.061084
4,230594,2022-01-14 09:19:00,0.240089,260.306344,13.257986,18156.10,-2.863851,0.061080
5,230654,2022-01-14 09:20:00,0.247100,260.520560,13.257292,18169.90,-2.976081,0.061059
6,230687,2022-01-14 09:21:00,0.250528,260.560697,13.256597,18175.05,-3.030952,0.061078
7,230752,2022-01-14 09:22:00,0.250004,260.558286,13.255903,18174.55,-3.022556,0.061069
8,230800,2022-01-14 09:23:00,0.258442,260.741460,13.255208,18188.90,-3.157570,0.061080


In [63]:
contract = 'NIFTY22JAN18750CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,4500,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 4876
result rsquared:  0.7769805898314962


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,295733,2022-01-19 09:16:00,0.050851,206.242048,8.260069,18046.00,0.156974,0.060760
1,295800,2022-01-19 09:17:00,0.053778,206.396553,8.259375,18061.30,0.109100,0.060665
2,295866,2022-01-19 09:18:00,0.055525,206.457804,8.258681,18068.60,0.080563,0.060610
3,295896,2022-01-19 09:19:00,0.053752,206.311321,8.257986,18057.00,0.109521,0.060665
4,295967,2022-01-19 09:20:00,0.053276,206.410877,8.257292,18063.00,0.117306,0.060681
5,296008,2022-01-19 09:21:00,0.057641,206.542990,8.256597,18079.00,0.046052,0.060548
6,296067,2022-01-19 09:22:00,0.057036,206.597998,8.255903,18082.00,0.055924,0.060566
7,296110,2022-01-19 09:23:00,0.056755,206.683123,8.255208,18087.45,0.060508,0.060575
8,296197,2022-01-19 09:24:00,0.055160,206.596405,8.254514,18080.00,0.086527,0.060623


In [65]:
contract = 'NIFTY22JAN19000CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,5242,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5252
result rsquared:  0.48242336694214805


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,319862,2022-01-19 15:22:00,0.024314,201.961748,8.005903,17975.30,0.607489,0.062608
1,319942,2022-01-19 15:23:00,0.024081,201.935784,8.005208,17973.30,0.611205,0.062610
2,320006,2022-01-19 15:24:00,0.023946,201.995342,8.004514,17977.20,0.613360,0.062615
3,320086,2022-01-19 15:25:00,0.022897,202.014253,8.003819,17975.00,0.630095,0.062632
4,320135,2022-01-19 15:26:00,0.023129,202.017968,8.003125,17976.70,0.626402,0.062629
5,320200,2022-01-19 15:27:00,0.023292,201.962056,8.002431,17974.25,0.623792,0.062624
6,320260,2022-01-19 15:28:00,0.023926,201.942934,8.001736,17976.00,0.613679,0.062615
7,320317,2022-01-19 15:29:00,0.022905,201.986367,8.001042,17975.50,0.629967,0.062632
8,320414,2022-01-19 15:30:00,0.022905,201.977601,8.000347,17975.50,0.629967,0.062632


In [67]:
contract = 'NIFTY22JAN19250CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,4345,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 4357
result rsquared:  0.016883218596378202


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,319782,2022-01-19 15:20:00,0.017305,201.265950,8.007292,17964.95,-0.188123,0.005482
1,319830,2022-01-19 15:21:00,0.017898,201.265852,8.006597,17969.30,-0.212778,0.004527
2,319912,2022-01-19 15:22:00,0.017958,201.349042,8.005903,17975.30,-0.215239,0.004432
3,319968,2022-01-19 15:23:00,0.018120,201.290725,8.005208,17973.30,-0.221735,0.004187
4,320004,2022-01-19 15:24:00,0.018160,201.341795,8.004514,17977.20,-0.223327,0.004126
5,320069,2022-01-19 15:25:00,0.017955,201.318252,8.003819,17975.00,-0.215115,0.004436
6,320154,2022-01-19 15:26:00,0.017973,201.335527,8.003125,17976.70,-0.215814,0.004409
7,320225,2022-01-19 15:27:00,0.017948,201.289305,8.002431,17974.25,-0.214807,0.004448
8,320249,2022-01-19 15:28:00,0.018692,201.232265,8.001736,17976.00,-0.243927,0.003368


In [71]:
contract = 'NIFTY22JAN19500CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,4343,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 5238
result rsquared:  0.012965483059016458


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,264386,2022-01-17 13:06:00,0.014567,231.441890,10.100347,18323.00,0.644643,0.055239
1,264437,2022-01-17 13:07:00,0.014550,231.404029,10.099653,18321.10,0.644847,0.055234
2,264477,2022-01-17 13:08:00,0.014752,231.359664,10.098958,18320.00,0.642497,0.055283
3,264521,2022-01-17 13:09:00,0.014724,231.304320,10.098264,18317.00,0.642813,0.055274
4,264613,2022-01-17 13:10:00,0.014740,231.323225,10.097569,18318.70,0.642634,0.055279
5,264632,2022-01-17 13:11:00,0.014737,231.310532,10.096875,18318.40,0.642665,0.055278
6,264719,2022-01-17 13:12:00,0.014127,231.410375,10.096181,18321.60,0.650008,0.055144
7,264770,2022-01-17 13:13:00,0.014105,231.363252,10.095486,18319.10,0.650287,0.055138
8,264799,2022-01-17 13:14:00,0.014100,231.346674,10.094792,18318.55,0.650348,0.055137


In [144]:
contract = 'NIFTY22JAN19750CE'
tmp = Calls[Calls['opt_symbol']==contract]
print("available data rows:", tmp.count()[0])
NIFTY22JAN17250CE = generate_MV_delta(contract,0,840,True,10)
NIFTY22JAN17250CE.iloc[:,[2,7]]
NIFTY22JAN17250CE

available data rows: 851
result rsquared:  0.010963236800074716


,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,319879,2022-01-19 15:22:00,0.011400,199.622114,8.005903,17975.30,1.176354,0.084694
1,319929,2022-01-19 15:23:00,0.011391,199.579849,8.005208,17973.30,1.175718,0.084641
2,320029,2022-01-19 15:24:00,0.011409,199.636707,8.004514,17977.20,1.176960,0.084745
3,320081,2022-01-19 15:25:00,0.011712,199.514674,8.003819,17975.00,1.197993,0.086325
4,320110,2022-01-19 15:26:00,0.011720,199.534707,8.003125,17976.70,1.198572,0.086373
5,320209,2022-01-19 15:27:00,0.011708,199.484702,8.002431,17974.25,1.197738,0.086303
6,320254,2022-01-19 15:28:00,0.011717,199.505582,8.001736,17976.00,1.198333,0.086353
7,320359,2022-01-19 15:29:00,0.011714,199.488487,8.001042,17975.50,1.198163,0.086339
8,320433,2022-01-19 16:19:00,0.011714,199.055155,7.966319,17975.50,1.198163,0.086339


In [145]:
tmp

,Unnamed: 0,date,opt_price,volume_x,opt_symbol,fut_price,volume_y,fut_symbol,name,expiry,strike,instrument_type,ts,days_to_expiry,IV,delta,vega
258674,546214,2022-01-17 11:27:00+05:30,2.95,0.0,NIFTY22JAN19750CE,18319.50,6350.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-17 11:27:00,10.169097,0.168417,0.013711,231.141142
258728,546328,2022-01-17 11:28:00+05:30,2.95,0.0,NIFTY22JAN19750CE,18312.00,19150.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-17 11:28:00,10.168403,0.169220,0.013659,231.003671
258828,546546,2022-01-17 11:29:00+05:30,2.95,0.0,NIFTY22JAN19750CE,18309.60,9550.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-17 11:29:00,10.167708,0.169481,0.013642,230.954257
258848,546599,2022-01-17 11:30:00+05:30,2.95,0.0,NIFTY22JAN19750CE,18305.00,13250.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-17 11:30:00,10.167014,0.169976,0.013610,230.866696
258956,546817,2022-01-17 11:31:00+05:30,2.95,0.0,NIFTY22JAN19750CE,18305.05,11500.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-17 11:31:00,10.166319,0.169976,0.013610,230.859678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320254,683158,2022-01-19 15:28:00+05:30,2.95,0.0,NIFTY22JAN19750CE,17976.00,56600.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-19 15:28:00,8.001736,0.230957,0.011717,199.505582
320359,683406,2022-01-19 15:29:00+05:30,2.95,0.0,NIFTY22JAN19750CE,17975.50,38600.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-19 15:29:00,8.001042,0.231027,0.011714,199.488487
320433,683564,2022-01-19 16:19:00+05:30,2.95,0.0,NIFTY22JAN19750CE,17975.50,0.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-19 16:19:00,7.966319,0.231530,0.011714,199.055155
320490,683709,2022-01-19 16:30:00+05:30,2.95,0.0,NIFTY22JAN19750CE,17975.50,0.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19750.0,CE,2022-01-19 16:30:00,7.958681,0.231641,0.011714,198.959696


In [146]:
tmp.reset_index(inplace=True,drop=True)

In [149]:
idx = tmp.index[tmp['ts']<='2022-01-20 16:39:00'].tolist()[-1]
tmp.iloc[:idx-800].count()[0]

50

In [150]:
idx

850